* A formula of search is applied to OpenAlex to extract an equivalent database
* We load the inspire papers and extract the dois from the `recid`s
* We extract of the dois from the results in OpenAlex and compare with the dois from inspire:
  * `DD`: inspire dois
  * `FF`: OpenAlex dois
  * `FF-DD`: dois only in OpenAlex: see [Status analysis](#Status-analysis): `status` values are:
     * valid arXiv dois without extra dois: with `status['status'] = 'ok'` and `published = False`
     * new valid dois with `status['status'] = 'ok'` with `status['affiliations_in_inspire'] = False`,  `status['date_range'] = 'outside'`, `status['mathching_inspire_authors']`. The first fixes missing affiliations, the second may be outside the date range, the third are not in inspire but have inspire authors
     * duplicated entries in OpenAlex with `status in ['duplicated', 'duplicated_dois']`
     * erros in OpenAlex with `status in [wrong_countries, wrong_doi]`. The first are bad affilitions, the second is not the real doi 
     * extra works that maybe are outside the scope of inspire with `status = 'not_inspire_scope'`
     * `status = 'erratum_or_arxiv_doi'` ?
  * `DD-FF`: dois inly in inspire:

In [8]:
import requests
import json

In [9]:
LA = []
next_cursor = "*"
p = 200
url_topics = f"https://api.openalex.org/works?page=1&filter=topics.id:t10048|t10224|t10527|t10921|t11090|t10025|t10095|t11415|t11044|t11367|t10463,authors_count:1-10,authorships.countries:countries/co|countries/br|countries/ar|countries/cl|countries/mx|countries/pe|countries/ec|countries/uy|countries/bo|countries/cr|countries/ve|countries/cu|countries/do|countries/sv|countries/gt|countries/hn|countries/pa|countries/py,publication_year:1900-2022&sort=cited_by_count:desc&per_page={p}&cursor={next_cursor}"

In [10]:
loop = False
while loop:
    #url_topics = f"https://api.openalex.org/works?page=1&filter=authorships.author.id:a5049648126&sort=cited_by_count:desc&per_page=200&cursor={next_cursor}"
    
    j = requests.get(url_topics)
    
    la = j.json()
    
    la_meta = la.get('meta')
    next_cursor = la_meta.get('next_cursor')
    
    if not next_cursor:
        #break
        print('stop')
        break
    
    LA = LA + la.get('results')
    print(len(LA),end='\r')

In [11]:
if loop:
    with open("openalex_topics.json", "w") as json_file:
        json.dump(LA, json_file)
    print(len(LA))

In [12]:
with open('openalex_topics.json', 'r') as file:
        LA = json.load(file)

In [13]:
f = 'data/papers.json'
try:
    with open(f, 'r') as file:
        data = json.load(file)
        print("File data =", f)
except FileNotFoundError:
    print("Error: The file 'data.json' was not found.")
except json.JSONDecodeError:
    print("Error: Failed to decode JSON from the file.")

File data = data/papers.json


In [14]:
import pandas as pd

In [15]:
inspa = pd.DataFrame(data)

In [16]:
del data

In [17]:
import time

In [18]:
recids =inspa['recid'].unique()
len(recids)

49181

In [20]:
del inspa

In [21]:
393487 in recids

True

In [22]:
#list(recids).index(393487)

In [23]:
#recids = recids[32287:32288]

In [24]:
loop = False
if loop:
    dois = []
    for x in list(range(0,len(L)+1,80))[1:]:
        time.sleep(0.1)
        print(f"{x-80}:{x}",end='\r')
    
        q='%20or%20'.join([ f"recid:{x}" for x in recids[x-80:x]])
        
        url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=dois"
        
        jj = requests.get(url)
        
        try:
            LL = jj.json().get('hits').get('hits')
        except:
            LL = []
        
        if not LL:
            continue
            
        tmp = pd.DataFrame(LL)
        tmp['control_number'] = tmp['metadata'].str['control_number']
        tmp['doi'] = tmp['metadata'].str['dois'].str[0].str['value']
        dois = dois + tmp[['control_number','doi']].dropna().to_dict(orient='records')
        #break

In [25]:
if loop:
    with open("inspire_dois.json", "w") as json_file:
        json.dump(dois, json_file)

In [26]:
with open("inspire_dois.json", "r") as json_file:
    dois = json.load(json_file)

In [41]:
di = pd.DataFrame(dois)

In [42]:
di['doi'] = di['doi'].str.lower()

In [43]:
di['doi'].shape[0], di['doi'].unique().shape

(38713, (38713,))

In [ ]:
df = pd.DataFrame(LA)

In [31]:
del LA

In [45]:
df.shape

(41347, 51)

Primary topics large that study area topics (11)

In [46]:
len(df['primary_topic'].str['display_name'].unique())

251

In [47]:
topics = url_topics.split('topics.id:')[-1].split(',')[0].split('|')
topics = [t.lower() for t in topics]

In [48]:
def topics_score(L,topics,score = 0):
    LL = []
    for d in L:
        if d['id'].split('/')[-1].lower() in topics and d['score'] > score:
            LL.append(d)
    return LL

In [49]:
df['filtered_topics'] = df['topics'].apply(lambda L: topics_score(L,topics,score=0))

In [50]:
df.shape

(41347, 51)

In [51]:
score = 0.9
df = df[df['topics'].apply(lambda L: topics_score(L,topics,score=score)).apply(len)> 0].reset_index(drop=True)
df.shape

(41347, 51)

In [52]:
df['doi'] = df['doi'].str.lower().str.split('https://doi.org/').str[-1]
df['doi'].shape,df['doi'].dropna().shape,df['doi'].dropna().unique().shape

((41347,), (38995,), (38993,))

In [53]:
DD = di['doi'].dropna().to_list() # dois in inspire (missing dois if duplicates)
FF = df['doi'].dropna().to_list() # dois in OpenAlex

In [54]:
len(DD)

38713

In [55]:
len(FF)

38995

In [56]:
intersection = list(set(DD).intersection( FF ))
len(intersection) # In both inspire and OpenAlex

27138

In [57]:
dfa = df[df['doi'].apply(lambda d: d in intersection)].reset_index(drop=True)
authors = dfa['authorships'].apply(lambda L: [d.get('author').get('id').split('/')[-1] for d in L] )
authors = list(set([item for sublist in authors for item in sublist]))
len(authors) #authors both in inspire and OpenAlex

15527

In [58]:
DDD = list(set(DD).difference( FF  ))
FFF = list(set(FF).difference(DD))

In [59]:
len(DDD) # only inspire

11575

In [60]:
with open('DDD.json','w') as f:
    json.dump(DDD,f)

In [61]:
len(FFF) # only OpenAlex

11855

In [62]:
df[df['doi'].astype(str).str.contains('arxiv')].shape # OpenAlex works in arXiv only 

(1277, 51)

In [63]:
trueFF = df[df['primary_location'].str['source'].str['issn_l'].apply(lambda s: s is not None)]['doi'].dropna().to_list()

In [64]:
len(trueFF),len(FF)

(35136, 38995)

In [65]:
trueFFF = list(set(trueFF).difference(DD))

In [66]:
len(trueFFF)

8941

In [67]:
del df

In [68]:
del dfa

In [69]:
def get_inspire(url):
    jj = requests.get(url)
        
    try:
        LL = jj.json().get('hits').get('hits')
    except:
        LL = []

    return LL
    
def clean_dois(dois):
    return [d for d in dois if d.get('source') != 'arXiv' and d.get('material') != 'erratum']


def get_arxiv(doi,recids):
    status = {}
    oadoi = None
    q = '/'.join(doi.split('/')[1:]).replace('arxiv.','arxiv:')
    url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=control_number,dois"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL:
        try:
            dois = clean_dois(LL[0].get('metadata').get('dois'))
            oadoi = dois[0].get('value').lower()
        except:
            oadoi = None
    
    if oadoi:
        url = f"https://api.openalex.org/works?filter=doi:{oadoi}"
        jj = requests.get(url)
        time.sleep(0.1)
    
        if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
            status = {'status':'duplicated',
             'arXiv': doi,
             'doi': oadoi
            }
            return status

    if LL:
        recid = LL[0].get('metadata').get('control_number')
        if recid in recids:
            status = {'status': 'not_doi','recid': recid, 'doi': doi}
        else:
            status = {'status': 'check_countries', 'doi': doi} # arXiv without doi

    return status

def get_dois(doi,DD):
    status = {}
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=control_number,authors,dois,publication_info"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL and LL[0].get('metadata').get('dois'):
        dois = LL[0].get('metadata').get('dois')
        values_doi = [d.get('value') for d in dois]
        if doi in values_doi and len(values_doi) > 1:
            all_dois = LL[0].get('metadata').get('dois')
            dois = clean_dois(all_dois) # Clean unwanted dois
            new_doi = [d.get('value').lower() for d in dois if d.get('value') != doi]
            if new_doi:
                new_doi = new_doi[0]
                if new_doi in DD:
                    status = {'status': 'duplicated_dois', 'new_doi': new_doi, 'doi': doi, 'dois': all_dois}
                else:
                    status = {'status': 'wrong_doi','new_doi': new_doi, 'doi': doi, 'dois': all_dois}
            else:
                recid = LL[0].get('metadata').get('control_number')
                publication_info = LL[0].get('metadata').get('publication_info')
                in_recids = False
                if recid in recids:
                    in_recids = True    
                # bad doi was erratum or arXiv                
                status = {'status': 'erratum_or_arxiv_doi_or_out_of_range', 
                          'doi': doi, 
                          'dois': all_dois,
                          'recid': recid,
                          'in_recids': in_recids,
                          'publication_info': publication_info}

    return status

def get_countries(doi):
    status = {}
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=authors,dois"
    LL = get_inspire(url)
    time.sleep(0.1)

    if LL:
        try:
            affiliations = [d for d in LL[0].get('metadata').get('authors') if d.get('affiliations')]
        except:
            affiliations = []
        
        if affiliations:
            status = {'status': 'wrong_countries', 'doi': doi} # right in inspirehep
        else:
            status = {'status': 'right_countries', 'doi': doi} # missing in inspirehep
            
    return status

def get_authors(doi,authors):
    status = {}
    url = f"https://api.openalex.org/works?filter=doi:{doi}"
    jj = requests.get(url)
    time.sleep(0.1)
    if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
        try:
            au = [ d.get('author').get('id').split('/')[-1] for d in jj.json().get('results')[0].get('authorships')]
        except:
            au = []
        auint = list(set(au).intersection(authors))    
        if len(auint) > 0:
            status = { 'status': 'ok',
                       'mathching_inspire_authors': True,
                       'mathching_inspire_author_ids': auint,
                       'doi': doi}
        else: # outside inspire
            status = { 'status': 'not_inspire_scope', 
                       'mathching_inspire_authors': False,
                       'mathching_inspire_author_ids': [],
                       'doi': doi}
    return status

In [70]:
no_issn_dois = list(set(FFF).difference(trueFFF)) # doi not relate to journals with issns
no_issn_dois[0:15]

['10.1109/plasma.1994.588706',
 '10.22323/1.301.0130',
 '10.1007/10856495_44',
 '10.22323/1.416.0016',
 '10.11606/d.43.2022.tde-16052022-124208',
 '10.1007/978-1-4684-3716-4_38',
 '10.48550/arxiv.gr-qc/9509041',
 '10.5281/zenodo.5718758',
 '10.1142/9789812810922_0010',
 '10.48550/arxiv.1605.05592',
 '10.48550/arxiv.hep-ph/9906328',
 '10.1142/9789813226609_0170',
 '10.1109/sbmicro.2015.7298138',
 '10.22323/1.174.0058',
 '10.1109/imoc.1999.866138']

## Status analysis

In [71]:
2151035 not in recids

True

In [75]:
def get_year(arxiv_doi):
    arxiv_id =arxiv_doi.split('/')
    if len(arxiv_id) == 2:
        year = eval('20'+arxiv_id[-1].split('arxiv.')[-1][:2])
    elif len(arxiv_id) == 3:
        year = arxiv_id[-1][:2]
        if year[0] == '0':
             year = eval('20' + year)
        else:
            year = eval('19' + year)
    
    return year

In [76]:
max_year = 2022
arxiv_doi = '10.48550/arxiv.0801.2559'
arxiv_doi = '10.48550/arxiv.astro-ph/9505300'
arxiv_doi = '10.48550/arxiv.astro-ph/0505300'
get_year(arxiv_doi)

2005

In [77]:
doi = '10.1088/1742-6596/590/1/012008'
doi =  '10.48550/arxiv.0801.2559'
#doi = '10.1103/physrevd.93.124020'
#doi = '10.48550/arxiv.astro-ph/0505300' # test: not_doi
#doi = '10.48550/arxiv.1909.10299' # test: duplicated
#doi = '10.1007/s100520050258' # test: duplicated_dois
#doi = '10.1140/epjc/s10052-015-3288-x' # test: right_countries
#doi = '10.1007/bf02817915' # test: wrong_countries
doi = '10.24275/uami.gq67jr29s' # test: 'mathching_inspire_authors' and 'status': 'not_inspire_scope'
doi = '10.1016/0003-4916(82)90173-7' # test matching_authors
doi = '10.1063/1.2710571' # test wrong_affiliations
doi = '10.1016/0550-3213(95)00620-6' # test duplicated dois
doi = '10.1088/1742-6596/590/1/012008' #test
doi = '10.1140/epjc/s10052-022-11127-w' # test outside_range
doi = '10.1007/bf00762333' # test 'mathching_inspire_authors'

In [78]:
def check_doi(doi, recids, DD, authors, max_year):
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=control_number,authors,dois,publication_info"
    LL = get_inspire(url)
    time.sleep(0.1)

    
    status = {'doi':doi,'in_inspire': False}
    arxiv_doi = None
    if doi.find('/arxiv.') == 8:
        arxiv_doi = doi
    
    status['arxiv_doi'] = arxiv_doi
    status['in_inspire'] = None
    
    metadata = None
    if LL:
        metadata = LL[0].get('metadata')
        if metadata:
            status['in_inspire'] = True
    inspire_authors = None
    all_dois = None
    control_number = None
    publication_info = None
    if metadata:
        inspire_authors = metadata.get('authors')
        all_dois = metadata.get('dois')
        control_number = metadata.get('control_number')
        publication_info = metadata.get('publication_info')
    
    inspire_dois = []
    status['recid'] = control_number
    status['in_recids'] = False
    status['published'] = True
    status['erratum_doi'] = None
    status['inspire_dois'] = []
    status['new_doi'] = None
    status['oa_errors'] = []
    status['inspire_scope'] = True
    status['new_doi_in_inspire'] = None
    status['affiliations_in_inspire'] = None
    status['mathching_inspire_authors'] = None
    status['mathching_inspire_author_ids'] = None
    status['date_range'] = None
    status['year'] = None
    
    if publication_info:
        try:
            status['year'] = max([d.get('year') for d in publication_info])
        except:
            status['year'] = None
    
    if not status['year'] and status['arxiv_doi']:
        status['year'] = get_year(status['arxiv_doi'])
    
    # chek if already in database
    if control_number and control_number in recids:
        status['in_recids'] = True
    else:
        # check inspire_scope
        pass
    
    # Check if published with doi
    if all_dois:
        erratum_doi = [d for d in all_dois if d.get('material') == 'erratum']
        if erratum_doi:
            status['erratum_doi'] = erratum_doi[0]
        inspire_dois = [d for d in all_dois if d.get('source') != 'arXiv' and d.get('material') != 'erratum']
        status['inspire_dois'] = list(set([d.get('value').lower() for d in inspire_dois]))
    else:
        if status['in_inspire']:
            status['published'] = False
    
    # Analysis
    status['status'] = None
    if status.get('arxiv_doi') and status['inspire_dois']:
        status['new_doi'] = status['inspire_dois'][0]
        status['status'] = 'duplicated'
        status['oa_errors'].append(status['status'])
    else:
        status['status'] = 'ok'
    
    if not status.get('arxiv_doi') and status['inspire_dois']:
        if len(status['inspire_dois']) > 1:
            status['new_doi'] = [x for x in status['inspire_dois'] if x != doi ][0]
            status['status'] = 'duplicated_dois'
            status['oa_errors'].append(status['status'])
    else:
        if not status['oa_errors']:
            status['status'] = 'ok'
    
    if status['new_doi'] and status['new_doi'] in DD:
        status['new_doi_in_inspire'] = True
    else:
        status['new_doi_in_inspire'] = False
    if inspire_authors:
        affiliations = [d for d in inspire_authors if d.get('affiliations')]
        if affiliations: # not affiliations in inspire
            status['affiliations_in_inspire'] = True
        else:
            status['affiliations_in_inspire'] = False
    
    # not in inspire LA database but in inspire → affiliations exist and are not LA
    if status['in_inspire'] and not status['in_recids'] and status['affiliations_in_inspire']:
        # check year conflict
        if status['year'] and status['year'] >= max_year:
            status['status'] = 'ok'
            status['date_range'] = 'outside'
        else:
            status['status'] = 'wrong_countries'
            status['oa_errors'].append(status['status'])
    else:
        if not status['oa_errors']:
            status['status'] = 'ok'
    
    # not in inspire → check for inspire authors
    if not status['in_inspire'] and not status['in_recids']:
        check_status = get_authors(doi,authors)
        status['status'] = check_status['status']
        if status['status'] == 'not_inspire_scope':
            status['oa_errors'].append(status['status'])
        else:
            if not status['oa_errors']:
                status['status'] = 'ok'
            
        status['mathching_inspire_authors'] = check_status['mathching_inspire_authors']
        status['mathching_inspire_author_ids'] = check_status['mathching_inspire_author_ids']
    
    
    return status

In [79]:
#status

In [80]:
#dfa[dfa['authorships'].apply(lambda L: [d.get('author').get('id').split('/')[-1] for d in L] ).apply(lambda L: 'A5058759801' in L)]

In [81]:
#check_doi(doi,recids, DD, authors, max_year)

In [82]:
len(FFF)

11855

In [94]:
kk = [0,1,2,3,4,5]

In [98]:
kk[:3]

[0, 1, 2]

In [96]:
len(kk[:3])

3

In [99]:
kk[3:]

[3, 4, 5]

In [86]:
len(ST)

8967

In [100]:
#ST = []
i = 0
for doi in FFF[8967:]:
    status = {}
    i += 1
    print(str(i).zfill(5),end='\r',flush=True)
    status = check_doi(doi,recids, DD, authors, max_year)
    ST.append(status)

In [104]:
with open('only_openalex.json','w') as f:
    json.dump(ST,f)

In [102]:
len(FFF)

11855

In [105]:
len(DDD)

11575

In [106]:
DDD[:10]

['10.1142/9789814374552_0382',
 '10.1103/physrevc.101.044314',
 '10.1103/physrevd.104.029901',
 '10.1142/s0217751x05022305',
 '10.1038/223485b0',
 '10.22323/1.174.0149',
 '10.22323/1.031.0001',
 '10.1103/physrevd.90.025019',
 '10.1006/aphy.1996.0079',
 '10.1088/0264-9381/3/5/016']

In [ ]:
OA = []
i = 0
for t in DDD:
    url = f"https://api.openalex.org/works?filter=doi:{t}"
    j = requests.get(url)
    try:
        jj = j.json().get('results')[0]
    except:
        pass
    if jj:
        OA.append(jj)
    print(str(i).zfill(5),end='\r')
    i += 1
    time.sleep(0.1)

In [109]:
OA

[{'id': 'https://openalex.org/W2316811290',
  'doi': 'https://doi.org/10.1142/9789814374552_0382',
  'title': 'OSTROGRADSKI APPROACH FOR THE REGGE–TEITELBOIM MODEL',
  'display_name': 'OSTROGRADSKI APPROACH FOR THE REGGE–TEITELBOIM MODEL',
  'publication_year': 2012,
  'publication_date': '2012-02-01',
  'ids': {'openalex': 'https://openalex.org/W2316811290',
   'doi': 'https://doi.org/10.1142/9789814374552_0382',
   'mag': '2316811290'},
  'language': 'en',
  'primary_location': {'is_oa': False,
   'landing_page_url': 'https://doi.org/10.1142/9789814374552_0382',
   'pdf_url': None,
   'source': None,
   'license': None,
   'license_id': None,
   'version': None,
   'is_accepted': False,
   'is_published': False},
  'type': 'article',
  'type_crossref': 'proceedings-article',
  'indexed_in': ['crossref'],
  'open_access': {'is_oa': False,
   'oa_status': 'closed',
   'oa_url': None,
   'any_repository_has_fulltext': False},
  'authorships': [{'author_position': 'first',
    'author': 

In [390]:
ST = []
i = 0
for doi in FFF[0:20]:
    status = {}
    i += 1
    print(str(i).zfill(5),end='\r',flush=True)

    #status = get_recid(doi,recids)
    #if status:
    #    ST.append(status)
    #    #pass
    #    continue
    
    
    if doi.find('/arxiv.') == 8:
        status = get_arxiv(doi,recids)
    if status:
        ST.append(status)
        #pass
        continue

    status = get_countries(doi)
    if status:
        ST.append(status)
        #pass
        continue

    
    status = get_dois(doi,DD)
    if status:
        ST.append(status)
        #pass
        continue

    status = get_authors(doi,authors)
    ST.append(status)
    #if i%100 == 0:


In [367]:
doi

'10.24275/uami.gq67jr29s'

In [391]:
ST

[{'status': 'right_countries', 'doi': '10.1063/1.2710571'},
 {'status': 'wrong_countries', 'doi': '10.1007/s100529800817'},
 {'status': 'not_doi', 'recid': 1318327, 'doi': '10.48550/arxiv.1409.5695'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/stu2558'},
 {'status': 'not_inspire_scope', 'doi': '10.24275/uami.gq67jr29s'},
 {'status': 'not_inspire_scope', 'doi': '10.2172/434870'},
 {'status': 'wrong_countries', 'doi': '10.1007/bf02817915'},
 {'status': 'wrong_countries', 'doi': '10.1016/0550-3213(95)00620-6'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/sty733'},
 {'status': 'duplicated',
  'arXiv': '10.48550/arxiv.2007.05004',
  'doi': '10.1103/physrevd.107.095030'},
 {'status': 'mathching_inspire_authors',
  'authors_id': ['A5053412812', 'A5068807109'],
  'doi': '10.1016/0003-4916(82)90173-7'},
 {'status': 'wrong_countries', 'doi': '10.1140/epjc/s10052-022-11127-w'},
 {'status': 'not_inspire_scope', 'doi': '10.1109/tasc.2022.3157659'},
 {'status': 'right_countries', 

In [258]:
1331731 in recids

False

In [183]:
'10.1016/0550-3213(95)00620-6' in DD

False

In [217]:
'10.1016/0550-3213(95)00259-u' in DD

True

In [116]:
doi = '10.48550/arxiv.1409.5695'

In [127]:
    status = {}
    oadoi = None
    q = '/'.join(doi.split('/')[1:]).replace('arxiv.','arxiv:')
    url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=control_number,dois"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL:
        try:
            oadoi = [ d for d in LL[0].get('metadata').get('dois') if d.get('source') != 'arXiv' and d.get('material') != 'erratum' 
              ][0].get('value').lower()
        except:
            oadoi = None
    
    if oadoi:
        url = f"https://api.openalex.org/works?filter=doi:{oadoi}"
        jj = requests.get(url)
        time.sleep(0.1)
    
        if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
            status = {'status':'duplicated',
             'arXiv': doi,
             'doi': oadoi
            }
            #return status

    recid = LL[0].get('metadata').get('control_number')
    if LL:
        if recid in recids:
            status = {'status': 'not_doi','recid': recid, 'doi': doi}
        else:
            status = {'status': 'check_countries', 'doi': doi} # arXiv without doi

In [128]:
status

{'status': 'not_doi', 'recid': 1318327, 'doi': '10.48550/arxiv.1409.5695'}

In [120]:
LL[0].get('metadata').get('control_number')

1318327

In [87]:
ST

[{'status': 'right_countries', 'doi': '10.1063/1.2710571'},
 {'status': 'wrong_countries', 'doi': '10.1007/s100529800817'},
 {'status': 'check_countries', 'doi': '10.48550/arxiv.1409.5695'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/stu2558'},
 {'status': 'not_mathching_authors', 'doi': '10.24275/uami.gq67jr29s'},
 {'status': 'not_mathching_authors', 'doi': '10.2172/434870'},
 {'status': 'wrong_countries', 'doi': '10.1007/bf02817915'},
 {'status': 'wrong_countries', 'doi': '10.1016/0550-3213(95)00620-6'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/sty733'},
 {'status': 'duplicated',
  'arXiv': '10.48550/arxiv.2007.05004',
  'doi': '10.1103/physrevd.107.095030'}]

In [83]:
get_countries(doi)

{'status': 'wrong_countries', 'doi': '10.1007/s100529800817'}

In [73]:
i += 1
print(str(i).zfill(5),end='\r')
 